In [23]:
# Libs necessairas
import pandas as pd
import numpy as np

# Analises gráficas
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# lib para http
import requests

# Lib para minerar os dados da pagina
from bs4 import BeautifulSoup

# Lib para daras
from datetime import datetime

# Avisos
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Site dos dados
Url = 'https://pt.wikipedia.org/wiki/Lista_de_pa%C3%ADses_por_PIB_nominal'

print( Url )

https://pt.wikipedia.org/wiki/Lista_de_pa%C3%ADses_por_PIB_nominal


In [3]:
# Scraping dos dados
Tabelas_Webs = pd.read_html(Url)

len(Tabelas_Webs)

11

In [4]:
# Tabela
Tabela_PIB = Tabelas_Webs[3]

# Dimensão
Tabela_PIB.shape

(196, 3)

In [5]:
# Verificando Campos Nulos
Tabela_PIB.isnull().sum()

Posição                  5
País                     0
PIB (Trilhões de US$)    1
dtype: int64

In [7]:
# Verificando colunas
Tabela_PIB.dtypes

Posição                   object
País                      object
PIB (Trilhões de US$)    float64
dtype: object

In [8]:
# Verificar nulos
Tabela_PIB.loc[ Tabela_PIB['Posição'].isnull() ]

,Posição,País,PIB (Trilhões de US$)
0,NaN,Mundo[22],84929.508
2,NaN,União Europeia,18736.855
37,NaN,Hong Kong,362721.000
65,NaN,Porto Rico,101131.000
84,NaN,Macau,54545.000


In [13]:
# Ajustando o valor da coluna
Tabela_PIB['PIB (Trilhões de US$)'] = pd.to_numeric( Tabela_PIB['PIB (Trilhões de US$)'] )

# Filtrar posições não nulos
Filtro = Tabela_PIB.loc[ Tabela_PIB['Posição'].notnull() ]

# Verificando
Filtro.head(10)

,Posição,País,PIB (Trilhões de US$)
1,1,Estados Unidos,20580250.0
3,2,China,14860775.0
4,3,Japão,4910580.0
5,4,Alemanha,3780553.0
6,5,Reino Unido,2638296.0
7,6,Índia,2592583.0
8,7,França,2551451.0
9,8,Itália,1848222.0
10,9,Canadá,1600264.0
11,10,Coreia do Sul,1586786.0


In [14]:
# Conexão com o site
Pagina_Web = requests.get(Url)

# BS4 vai intrepretar o arquivo html
Coleta = BeautifulSoup(Pagina_Web.text, 'html.parser')

# Tipo
type( Coleta )

bs4.BeautifulSoup

In [16]:
# Capturar todos os links
Imagens = Coleta.find_all('img', attrs={'class' : 'thumbborder'} )

# Salvando o slinks
Links = []

# Loop ajustando a imagem
for Link in Imagens:
    Link_Imagem = 'https:' + Link['src']
    Links.append( Link_Imagem )
    
print(f'Foram localizado {len(Links)} links de Imagens')

Foram localizado 594 links de Imagens


In [20]:
# Verificando
print( Links[0] )
print( Links[1] )
print( Links[2] )

https://upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Flag_of_the_United_States.svg/22px-Flag_of_the_United_States.svg.png
https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Flag_of_Europe.svg/22px-Flag_of_Europe.svg.png
https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/22px-Flag_of_the_People%27s_Republic_of_China.svg.png


In [24]:
# Lib para imagens
from PIL import Image
from io import BytesIO

# Pegando um link de imagem
Response = requests.get(Links[1] )

# Convertendo para imagen
Image.open( BytesIO(Response.content) )

In [25]:
# Removendo a bandeira da UE
Links.pop(1)

'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Flag_of_Europe.svg/22px-Flag_of_Europe.svg.png'

In [33]:
# Iniciar a figura do gráfico

# Titulos
Titulos = [ 'Top 10 PIBs Mundiais' ]
Rodape = 'Análise por: @Odemir Depieri Jr'

# Initialize figure with subplots
Figura = make_subplots(
    rows=1, cols=1, subplot_titles=(Titulos) )

# Titulo e Tamanho
Figura.update_layout( 
    title_text='Lista do Fundo Monetário Internacional', 
    height=500,
    #template='plotly_dark',
    showlegend=False,
    title={
        'y':0.97,'x':0.5,
        'xanchor': 'center','yanchor': 'top'},
    )

# Grafico 1
Figura.add_trace(
    go.Bar( 
        x=Filtro['País'].head(10), 
        y=Filtro['PIB (Trilhões de US$)'].head(10),
        text=Filtro['PIB (Trilhões de US$)'].head(10),
        textposition='auto',
    ), 
        row=1, col=1 )

# Ajustando o numero
Figura.update_traces( texttemplate='%{text:.3s}', textposition='outside')

# Incluindo as bandeiras
Posicao_Esquerda = 0

# Loop nas bandeiiras
for Loop in Links[0:10]:
    
    # Carregando o link da imagem
    Link_Imagenm = requests.get( Loop )
    
    # Convertendo para imagem
    Bandeira = Image.open( BytesIO(Link_Imagenm.content) )

    # Imagem
    Figura.add_layout_image(
      dict(
        source=Bandeira,
        x=0.062 + Posicao_Esquerda, 
        y= -0.15,
        sizex=0.05, sizey=0.05,
        xanchor='right', yanchor='bottom'
      )
    )
    
    # Ajustando a imagem
    Posicao_Esquerda += 0.1

# Plot
Figura.show()

In [61]:
# Importando a biblioteca
import win32com.client as win32

# Instanciado o aplitivo na máquina
Outlook = win32.Dispatch('outlook.application')

# Criando o e-mail
Email = Outlook.CreateItem(0)

# Destino
Email.To = 'data.vikingx@gmail.com; data.vikingx@gmail.com; data.vikingx@gmail.com; data.vikingx@gmail.com'
# Copia
Email.CC = 'data.vikingx@gmail.com'
# Copia oculta
Email.BCC = ''

# Assunto
Email.Subject = 'E-mail com Python.'

# Corpo do e-mail
Email.Body = 'Message body'

Local_Imagem = r'C:\Users\datav\Downloads\Indicador.png'
Email.Attachments.Add( Local_Imagem )

Codigo_HTML = '''
<h3> Relátorio PIBs mundiais </h3>
<p> Segue análise dos valores em 2020 </p>
<img src="Indicador.png" width="50%" alt="Indicador"/>
<br>
<br>
<p> 
    Qualquer dúvida estou a disposição <br> 
    <i> Odemir Depieri Jr </i>
</p>
'''

# Aplicando HTML no seu e-mail
Email.HTMLBody = Codigo_HTML
Email.Display(False)

# Enviar e-mail
#Email.Send()      